# Prerequisite

In [3]:
import pathlib

import pandas as pd

pd.set_option("display.max_columns", 20)

DATA_PATH = pathlib.Path("../../data/titanic.csv")

data = pd.read_csv(DATA_PATH)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [5]:
data.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [6]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# 在原数据的基础上进一步挖掘信息

In [7]:
# Age Group
def classify_age(age):
    if 0 < age <= 6:
        return "Chilren"
    elif 6 < age <= 17:
        return "Teenagers"
    elif 17 < age <= 40:
        return "Youth"
    elif 40 < age <= 60:
        return "Middle-aged"
    elif 60 < age <= 100:
        return "Elderly"
    else:
        return "Unknown"


data["AgeCategory"] = data["Age"].map(classify_age)
data["AgeCategory"].value_counts()

Youth          451
Unknown        177
Middle-aged    128
Teenagers       66
Chilren         47
Elderly         22
Name: AgeCategory, dtype: int64

In [8]:
# Pclass stats
data["Pclass"].value_counts(sort=False)

3    491
1    216
2    184
Name: Pclass, dtype: int64

In [9]:
# Fare stats
data["Fare"].describe()

count    891.000000
mean      32.204208
std       49.693429
min        0.000000
25%        7.910400
50%       14.454200
75%       31.000000
max      512.329200
Name: Fare, dtype: float64

In [10]:
grp = data.loc[data["Fare"] >= 1].groupby("Pclass")["Fare"].describe()
grp = grp.drop("mean", axis=1)
grp

,count,std,min,25%,50%,75%,max
Pclass,,,,,,,
1,211.0,78.212155,5.0000,32.9104,61.9792,99.9625,512.3292
2,178.0,13.083169,10.5000,13.0000,15.0229,26.0000,73.5000
3,487.0,11.760718,4.0125,7.7500,8.0500,15.5000,69.5500


In [11]:
data["FareLevel"] = pd.cut(
    data["Fare"], bins=[0, 15, 60, 600], labels=["normal", "middle", "upper"]
)
data["FareLevel"].value_counts()

normal    443
middle    311
upper     122
Name: FareLevel, dtype: int64

# 无用和缺失数据的处理

In [12]:
# Cabin
data = data.drop("Cabin", axis=1)
print(data.columns)

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Embarked', 'AgeCategory', 'FareLevel'],
      dtype='object')


In [13]:
# Ticket
data = data.drop("Ticket", axis=1)
print(data.columns)

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Fare', 'Embarked', 'AgeCategory', 'FareLevel'],
      dtype='object')


In [14]:
import numpy as np

data_with_na = pd.DataFrame(
    dict(
        a=[1, np.nan, 2, 4],
        b=[np.nan, "a", np.nan, "c"],
    )
)
print(data_with_na)

     a    b
0  1.0  NaN
1  NaN    a
2  2.0  NaN
3  4.0    c


In [15]:
data_with_na.fillna(0)

,a,b
0,1.0,0
1,0.0,a
2,2.0,0
3,4.0,c


In [16]:
data_with_na.fillna({"a": 0, "b": "missing"})

,a,b
0,1.0,missing
1,0.0,a
2,2.0,missing
3,4.0,c


In [17]:
data_with_na["a"].fillna(0)

0    1.0
1    0.0
2    2.0
3    4.0
Name: a, dtype: float64

In [18]:
data_with_na["b"].fillna("missing")

0    missing
1          a
2    missing
3          c
Name: b, dtype: object

In [19]:
# Dropping `Embarked` column NA value.

print(f"the rows before `dropna()`: {data.shape[0]}")
data = data.dropna(subset=["Embarked"])
print(f"the rows after `dropna()`: {data.shape[0]}")

the rows before `dropna()`: 891
the rows after `dropna()`: 889


# 脏数据和重复数据的处理

In [20]:
pat = r"(?P<FirstName>.*), (?P<Title>.*?)\. (?P<LastName>.*)"
data["Title"] = data["Name"].str.extract(pat, expand=True)["Title"]
data["Title"].value_counts()

Mr              517
Miss            181
Mrs             124
Master           40
Dr                7
Rev               6
Mlle              2
Major             2
Col               2
the Countess      1
Capt              1
Ms                1
Sir               1
Lady              1
Mme               1
Don               1
Jonkheer          1
Name: Title, dtype: int64

In [21]:
title_mapping = {
    "Mlle": "Miss",
    "Mme": "Mrs",
    "Ms": "Miss",
}
data["Title"] = data["Title"].replace(title_mapping)

In [22]:
title_stats = data["Title"].value_counts()
others = title_stats[title_stats < 10].index
print(others)

Index(['Dr', 'Rev', 'Major', 'Col', 'Don', 'Lady', 'Sir', 'Capt',
       'the Countess', 'Jonkheer'],
      dtype='object')


In [23]:
data["Title"] = data["Title"].replace(others, "Other")

In [24]:
data["Title"].value_counts()

Mr        517
Miss      184
Mrs       125
Master     40
Other      23
Name: Title, dtype: int64